# Create DEM Files
The final step would be to create actual DEM files and populate the DEM tindex with actual file locations.
Unfortunately, the computation requirements of processing all the LAS files and producing over 1000 DEM tiles exceed
what is practical to perform within a notebook. Rather, this notebook provides some of the example methods for
processing the DEM tile index, locating the relevant LAS files by querying the LAS tile index, and creating DEM files
in different computing environments.

In [38]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np

### Load the LAS tile index

In [13]:
lti = gpd.read_file('../lidar/LAS_tindex.gpkg')
lti.head()

,location,srs,modified,created,geometry
0,adams/las_classified/1280_1930.las,EPSG:26972,2009-10-20T00:37:09,2018-12-13T21:46:57,POLYGON ((-91.47852667224002 40.17411279626138...
1,adams/las_classified/1156_2086.las,EPSG:26972,2009-10-19T22:34:02,2018-12-13T21:46:57,POLYGON ((-90.91659129758993 39.83871065542411...
2,adams/las_classified/2066_1216.las,EPSG:3444,2009-10-19T23:56:13,2018-12-13T21:46:57,"POLYGON ((-90.98977805630592 40.0029290734882,..."
3,adams/las_classified/1966_1234.las,EPSG:3444,2009-10-19T22:18:49,2018-12-13T21:46:57,POLYGON ((-91.34753807035025 40.04924836807506...
4,adams/las_classified/1240_1992.las,EPSG:26972,2009-10-19T23:15:51,2018-12-13T21:46:57,POLYGON ((-91.25494238878736 40.06662605018638...


### Load the DEM tile index

In [14]:
dti = gpd.read_file('../dem/DEM_tindex.gpkg')
dti.head()

,srs,zone,name,geometry
0,EPSG:3443,east,east_719_178,POLYGON ((-89.29442519763219 41.56450650515664...
1,EPSG:3443,east,east_719_174,POLYGON ((-89.29276436950822 41.45210485851405...
2,EPSG:3443,east,east_719_170,POLYGON ((-89.29111293732947 41.33970092370969...
3,EPSG:3443,east,east_719_166,POLYGON ((-89.28947084013322 41.22729470219819...
4,EPSG:3443,east,east_719_161,POLYGON ((-89.28783801753076 41.11488619546482...


### Locate LAS files overlapping DEM tile
Using an arbitrary DEM tile taken from the DEM tile index extract a list of LAS files necessary for creating the DEM file.

In [43]:
region = dti.iloc[20]
tiles = lti[lti.geometry.intersects(region['geometry'])]
print(f'{len(tiles)} overlapping LAS files found\n')
files = list(tiles['location'])
print(f'Relative path to first 10 LAS files: {files[:10]}')

436 overlapping LAS files found

Relative path to first 10 LAS files: ['fayette/las/730_800.las', 'fayette/las/738_788.las', 'fayette/las/748_780.las', 'fayette/las/752_800.las', 'fayette/las/746_776.las', 'fayette/las/748_802.las', 'fayette/las/732_774.las', 'fayette/las/762_786.las', 'fayette/las/750_786.las', 'fayette/las/736_778.las']


### Create a file with all the LAS files
Using the single DEM tile from the previous example, create a txt file with the absolute path to all the necessary
LAS files used to create a DEM tile. The file is named based on the *name* field of the DEM tile. This file is appropriate
for use as input to PDAL or pp2g.

In [41]:
fname = f'../{region["name"]}_las_files.txt'
absfiles = [os.path.join('/mnt/nrcs/isgs/lidar', f) for f in files]
with open(fname, 'w') as f:
    f.write('\n'.join(absfiles))
print(f'{fname} created with {len(absfiles)} files')

../east_719_759_las_files.txt created with 436 files


### Run PP2G using file list as input
PP2G is not installed within the notebook kernel so the following command will not work. It is provided as an
example of running pp2g from the command line.
```
$ mpirun -np 8 /app/p_points2grid -o dem/east_719_759.tif -m parallel -c 4 --mean --fill --fill_window_size=7 
  --bigtiff=1 --last_returns --output_format tif --resolution 5 -b 1000000 -t -f east_719_759_las_files.txt
```
